# Hugging Face Accelerate

`Accelerate` es una biblioteca de Hugging Face que permite ejecutar el mismo código PyTorch en cualquier configuración distribuida añadiendo sólo cuatro líneas de código.

## Instalación

Para instalar `accelerate` con `pip` simplemente ejecuta:

``` bash
pip install accelerate
```

Y con `conda`:

``` bash
conda install -c conda-forge accelerate
```

## Configuración

En cada entorno en el que se intale `accelerate` lo primero que se tiene que hacer es configurarlo, para ello ejecutamos en una terminal:

``` bash
accelerate config
```

In [1]:
!accelerate config

--------------------------------------------------------------------------------
In which compute environment are you running?
This machine
--------------------------------------------------------------------------------
multi-GPU
How many different machines will you use (use more than 1 for multi-node training)? [1]: 1
Should distributed operations be checked while running for errors? This can avoid timeout issues but will be slower. [yes/NO]: no
Do you wish to optimize your script with torch dynamo?[yes/NO]:no
Do you want to use DeepSpeed? [yes/NO]: no
Do you want to use FullyShardedDataParallel? [yes/NO]: no
Do you want to use Megatron-LM ? [yes/NO]: no
How many GPU(s) should be used for distributed training? [1]:2
What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:0,1
--------------------------------------------------------------------------------
Do you wish to use FP16 or BF16 (mixed precision)?
no
accelerate configuration saved at ~/

En mi caso las respuestas han sido
 * In which compute environment are you running?
    - [x] "This machine"
    - [_] "AWS (Amazon SageMaker)"
 > Quiero configurarlo en mi ordenador

 * Which type of machine are you using?
    - [_] multi-CPU
    - [_] multi-XPU
    - [x] multi-GPU
    - [_] multi-NPU
    - [_] TPU
 > Como tengo 2 GPUs y quiero ejecutar códigos distribuidos en ellas elijo `multi-GPU`
 
 * How many different machines will you use (use more than 1 for multi-node training)? [1]:
    - 1
 > Elijo `1` porque solo voy a ejecutar en mi ordenador

 * Should distributed operations be checked while running for errors? This can avoid timeout issues but will be slower. [yes/NO]:
    - no
 > Con esta opción, se puede elegir que `accelerate` chequee errores en la ejecución, pero haría que vaya más lento, así que elijo `no`, y en caso de que haya errores lo cambio a `yes`
 
 * Do you wish to optimize your script with torch dynamo?[yes/NO]:
    - no

 * Do you want to use FullyShardedDataParallel? [yes/NO]:
    - no
 
 * Do you want to use Megatron-LM ? [yes/NO]:
    - no
 
 * How many GPU(s) should be used for distributed training? [1]:
    - 2
 > Elijo `2` porque tengo 2 GPUs

 * What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:
    - 0,1
 > Elijo `0,1` porque quiero usar las dos GPUs

 * Do you wish to use FP16 or BF16 (mixed precision)?
    - [x] no
    - [_] fp16
    - [_] bf16
    - [_] fp8
 > De momento elijo `no`, porque para simplificar el código cuando no uso `acelerate` vamos a entrenar en fp32, pero lo ideal sería usar fp16

La configuración se guardará en `~/.cache/huggingface/accelerate/default_config.yaml` y se puede modificar en cualquier momento. Vamos a ver qué hay dentro

In [ ]:
!cat ~/.cache/huggingface/accelerate/default_config.yaml

compute_environment: LOCAL_MACHINE
debug: false
distributed_type: MULTI_GPU
downcast_bf16: 'no'
gpu_ids: 0,1
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false


Otra forma de ver la configuración que tenemos es ejecutando en una terminal:

``` bash
accelerate env
```

In [ ]:
!accelerate env


Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.28.0
- Platform: Linux-5.15.0-105-generic-x86_64-with-glibc2.31
- Python version: 3.11.8
- Numpy version: 1.26.4
- PyTorch version (GPU?): 2.2.1+cu121 (True)
- PyTorch XPU available: False
- PyTorch NPU available: False
- System RAM: 31.24 GB
- GPU type: NVIDIA GeForce RTX 3090
- `Accelerate` default config:
	- compute_environment: LOCAL_MACHINE
	- distributed_type: MULTI_GPU
	- mixed_precision: fp16
	- use_cpu: False
	- debug: False
	- num_processes: 2
	- machine_rank: 0
	- num_machines: 1
	- gpu_ids: 0,1
	- rdzv_backend: static
	- same_network: True
	- main_training_function: main
	- downcast_bf16: no
	- tpu_use_cluster: False
	- tpu_use_sudo: False
	- tpu_env: []


Una vez hemos configurado `accelerate` podemos probar si lo hemos hecho bien ejecutando en una terminal:

``` bash
accelerate test
```

In [ ]:
!accelerate test


Running:  accelerate-launch ~/miniconda3/envs/nlp/lib/python3.11/site-packages/accelerate/test_utils/scripts/test_script.py
stdout: **Initialization**
stdout: Testing, testing. 1, 2, 3.
stdout: Distributed environment: DistributedType.MULTI_GPU  Backend: nccl
stdout: Num processes: 2
stdout: Process index: 0
stdout: Local process index: 0
stdout: Device: cuda:0
stdout: 
stdout: Mixed precision type: fp16
stdout: 
stdout: Distributed environment: DistributedType.MULTI_GPU  Backend: nccl
stdout: Num processes: 2
stdout: Process index: 1
stdout: Local process index: 1
stdout: Device: cuda:1
stdout: 
stdout: Mixed precision type: fp16
stdout: 
stdout: 
stdout: **Test process execution**
stdout: 
stdout: **Test split between processes as a list**
stdout: 
stdout: **Test split between processes as a dict**
stdout: 
stdout: **Test split between processes as a tensor**
stdout: 
stdout: **Test random number generator synchronization**
stdout: All rng are properly synched.
stdout: 
stdout: **Da

Vemos que termina diciendo `Test is a success! You are ready for your distributed training!` por lo que todo está correcto.

## Entrenamiento

### Optimización del entrenamiento

#### Código base

Vamos a hacer primero un código de entrenamiento base y luego lo optimizaremos  para ver cómo se hace y cómo mejora

Primero vamos a buscar un dataset, en mi caso voy a usar el dataset [tweet_eval](https://huggingface.co/datasets/tweet_eval), que es un dataset de clasificación de tweets, en concreto voy a descargar el subset `emoji` que clasifica los tweets con emoticonos

In [ ]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emoji")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [ ]:
dataset["train"].info

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['❤', '😍', '😂', '💕', '🔥', '😊', '😎', '✨', '💙', '😘', '📷', '🇺🇸', '☀', '💜', '😉', '💯', '😁', '🎄', '📸', '😜'], id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='tweet_eval', config_name='emoji', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=3808792, num_examples=45000, shard_lengths=None, dataset_name='tweet_eval'), 'test': SplitInfo(name='test', num_bytes=4262151, num_examples=50000, shard_lengths=None, dataset_name='tweet_eval'), 'validation': SplitInfo(name='validation', num_bytes=396704, num_examples=5000, shard_lengths=None, dataset_name='tweet_eval')}, download_checksums={'hf://datasets/tweet_eval@b3a375baf0f409c77e6bc7aa35102b7b3534f8be/emoji/train-00000-of-00001.parquet': {'num_bytes': 2609973, 'checksum': None}, 'hf://datasets/tweet_eval@b3a375baf0f409c77e6bc7aa35

Vamos a ver las clases

In [ ]:
print(dataset["train"].info.features["label"].names)

['❤', '😍', '😂', '💕', '🔥', '😊', '😎', '✨', '💙', '😘', '📷', '🇺🇸', '☀', '💜', '😉', '💯', '😁', '🎄', '📸', '😜']


Y el número de clases

In [ ]:
num_classes = len(dataset["train"].info.features["label"].names)
num_classes

20

Vemos que el dataset tiene 20 clases

Vamos a ver la secuencia máxima de cada split

In [ ]:
max_len_train = 0
max_len_val = 0
max_len_test = 0

split = "train"
for i in range(len(dataset[split])):
    len_i = len(dataset[split][i]["text"])
    if len_i > max_len_train:
        max_len_train = len_i
split = "validation"
for i in range(len(dataset[split])):
    len_i = len(dataset[split][i]["text"])
    if len_i > max_len_val:
        max_len_val = len_i
split = "test"
for i in range(len(dataset[split])):
    len_i = len(dataset[split][i]["text"])
    if len_i > max_len_test:
        max_len_test = len_i

max_len_train, max_len_val, max_len_test

(142, 139, 167)

Así que definimos la secuencia máximo en general como 130 para la tokeniazción

In [ ]:
max_len = 130

A nosotros nos interesa el dataset tokenizado, no las secuencias en crudo, así que creamos un tokenizador

In [ ]:
from transformers import AutoTokenizer

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

Creamos una función de tokenización

In [ ]:
def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")

Y ahora tokenizamos el dataset

In [ ]:
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Como vemos ahora tenemos los tokens (`input_ids`) y las máscaras de atención (`attention_mask`), pero vamos a ver qué tipo de datos tenemos

In [ ]:
type(tokenized_dataset["train"][0]["input_ids"]), type(tokenized_dataset["train"][0]["attention_mask"]), type(tokenized_dataset["train"][0]["label"])

(list, list, int)

In [ ]:
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
type(tokenized_dataset["train"][0]["label"]), type(tokenized_dataset["train"][0]["input_ids"]), type(tokenized_dataset["train"][0]["attention_mask"])

(torch.Tensor, torch.Tensor, torch.Tensor)

Creamos un dataloader

In [ ]:
import torch
from torch.utils.data import DataLoader
BS = 64

dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

Cargamos el modelo

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)

Vamos a ver cómo es el modelo

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

Vamos a ver su última capa

In [ ]:
model.classifier.out_proj

Linear(in_features=768, out_features=2, bias=True)

In [ ]:
model.classifier.out_proj.in_features, model.classifier.out_proj.out_features

(768, 2)

Hemos visto que nuestro dataset tiene 20 clases, pero este modelo está entrenado para 2 clases, así que tenemos que modificar la última capa

In [ ]:
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)
model.classifier.out_proj

Linear(in_features=768, out_features=20, bias=True)

Ahora sí

Ahora creamos una función de loss

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()

Un optimizador

In [ ]:
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=5e-4)

Y por último una métrica

In [ ]:
import evaluate

metric = evaluate.load("accuracy")

Vamos a comprobar que está todo bien con una muestra

In [ ]:
sample = next(iter(dataloader["train"]))

In [ ]:
sample["input_ids"].shape, sample["attention_mask"].shape

(torch.Size([64, 130]), torch.Size([64, 130]))

Ahora esa muestra se la metemos al modelo

In [ ]:
model.to("cuda")
ouputs = model(input_ids=sample["input_ids"].to("cuda"), attention_mask=sample["attention_mask"].to("cuda"))
ouputs.logits.shape

torch.Size([64, 20])

Vemos que el modelo saca 64 batches, lo cual está bien, porque configuramos `BS = 20` y cada una con 20 salidas, lo cual está bien porque cambiamos el modelo para que a la salida de 20 valores

Obtenemos la de mayor valor

In [ ]:
predictions = torch.argmax(ouputs.logits, axis=-1)
predictions.shape

torch.Size([64])

Obtenemos la loss

In [ ]:
loss = loss_function(ouputs.logits, sample["label"].to("cuda"))
loss.item()

2.9990389347076416

Y el accuracy

In [ ]:
accuracy = metric.compute(predictions=predictions, references=sample["label"])["accuracy"]
accuracy

0.015625

Ya podemos crear un pequeño bucle de entrenamiento

In [ ]:
from fastprogress.fastprogress import master_bar, progress_bar

epochs = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

master_progress_bar = master_bar(range(epochs))
for i in master_progress_bar:
    model.train()
    progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        master_progress_bar.child.comment = f'loss: {loss}'

        loss.backward()
        optimizer.step()

    model.eval()
    progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
    master_progress_bar.main_bar.comment = f"Validation accuracy: {accuracy['accuracy']}\n"

#### Script con el código base

En la mayoría de la documentación de `accelerate` se explica cómo usar `accelerate` con scripts, así que de momento vamos a hacerlo así y al final explicaremos cómo hacerlo con un notebook

Primero vamos a crear una carpeta en la que vamos a guardar los scripts

In [1]:
!mkdir accelerate_scripts

Ahora escribimos el código base en un script

In [40]:
%%writefile accelerate_scripts/01_code_base.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from fastprogress.fastprogress import master_bar, progress_bar

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 64
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

master_progress_bar = master_bar(range(EPOCHS))
for i in master_progress_bar:
    model.train()
    progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        master_progress_bar.child.comment = f'loss: {loss}'

        loss.backward()
        optimizer.step()

    model.eval()
    progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
    master_progress_bar.main_bar.comment = f"Validation accuracy: {accuracy['accuracy']}\n"
print(f"Accuracy = {accuracy['accuracy']}")

Overwriting accelerate_scripts/01_code_base.py


Y ahora lo ejecutamos

In [41]:
%%time

!python accelerate_scripts/01_code_base.py

Accuracy = 0.2112                                                               
CPU times: user 2.12 s, sys: 391 ms, total: 2.51 s
Wall time: 3min 36s


Vemos que en mi ordenador ha tardado unos 3 minutos y medio

#### Código con accelerate

Ahora reemplazamos algunas cosas

 * En primer lugar importamos `Accelerator` y lo inicializamos

``` python
from accelerate import Accelerator
accelerator = Accelerator()
```

 * Ya no hacemos el típico

``` python 
torch.device("cuda" if torch.cuda.is_available() else "cpu")
```

 * Sino que dejamos que sea `acelerate` el que elija el dispositivo mediante

``` python
device = accelerator.device
```

 * Pasamos los elementos relevantes para el entrenamiento por el método `prepare` y ya no hacemos `model.to(device)`

``` python
model, optimizer, dataloader["train"], dataloader["validation"] = preprare(model, optimizer, dataloader["train"], dataloader["validation"])
```

 * Ya no mandamos los datos y el modelo a la GPU con `.to(device)` ya que `accelerate` se ha encargado de ello con el método `prepare`

 * En vez de hacer el backpropagation con `loss.backward()` dejamos que lo haga `accelerate` con
 
``` python
accelerator.backward(loss)
```

 * A la hora de calcular la métrica en el bucle de validación, necesitamos recopilar los valores de todos los puntos, en caso de estar haciendo un entrenamiento distribuido, para ello hacemos

``` python
predictions = accelerator.gather_for_metrics(predictions)
```

In [19]:
%%writefile accelerate_scripts/02_accelerate_base_code.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from fastprogress.fastprogress import master_bar, progress_bar

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 64
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

master_progress_bar = master_bar(range(EPOCHS))
for i in master_progress_bar:
    model.train()
    progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        master_progress_bar.child.comment = f'loss: {loss}'

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()
    print(f"End of training epoch {i}, outputs['logits'].shape: {outputs['logits'].shape}, labels.shape: {labels.shape}")

    model.eval()
    progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    print(f"End of validation epoch {i}, outputs['logits'].shape: {outputs['logits'].shape}, labels.shape: {labels.shape}")
    
    master_progress_bar.main_bar.comment = f"Validation accuracy: {accuracy['accuracy']}\n"

print(f"Accuracy = {accuracy['accuracy']}")

Overwriting accelerate_scripts/02_accelerate_base_code.py


Si te fijas he añadido estas dos líneas `print(f"End of training epoch {i}, outputs['logits'].shape: {outputs['logits'].shape}, labels.shape: {labels.shape}")` y la línea `print(f"End of validation epoch {i}, outputs['logits'].shape: {outputs['logits'].shape}, labels.shape: {labels.shape}")`, las he añadido aposta porque nos van a revelar algo muy importante

Ahora lo ejecutamos, para ejecutar los scripts de `accelerate` se hace con el comando `accelerate launch`

``` bash
accelerate launch script.py
```

In [29]:
%%time

!accelerate launch accelerate_scripts/02_accelerate_base_code.py

End of training epoch 0, outputs['logits'].shape: torch.Size([64, 20]), labels.shape: torch.Size([64])
End of training epoch 0, outputs['logits'].shape: torch.Size([64, 20]), labels.shape: torch.Size([64])
End of validation epoch 0, outputs['logits'].shape: torch.Size([64, 20]), labels.shape: torch.Size([8])
Accuracy = 0.206
End of validation epoch 0, outputs['logits'].shape: torch.Size([64, 20]), labels.shape: torch.Size([8])
Accuracy = 0.206
CPU times: user 1.6 s, sys: 272 ms, total: 1.88 s
Wall time: 2min 37s


Vemos que antes tardó unos 3 minutos y medio y ahora tarda más o menos 2 minutos y medio. Bastante mejora. Además si vemos los `print`s podemos ver que se han impreso dos veces.

¿Y esto cómo puede ser? pues porque `accelerate` ha paralelizado el entrenamiento en las dos GPUs que tengo, por lo que ha sido mucho más rápido.

Además, cuando ejecuté el primer script, esd ecir, cuando no usé `accelerate`, la GPU estaba casi llena, mientras que cuando he ejecutado el segundo, es decir, el que usa `accelerate`, las dos GPUs estaban muy poco utilizadas, por lo que podemos aumentar el batch size para intentar llenar las dos, vamos a ello!

In [27]:
%%writefile accelerate_scripts/03_accelerate_base_code_more_bs.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from fastprogress.fastprogress import master_bar, progress_bar

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

master_progress_bar = master_bar(range(EPOCHS))
for i in master_progress_bar:
    model.train()
    progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        master_progress_bar.child.comment = f'loss: {loss}'

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
    master_progress_bar.main_bar.comment = f"Validation accuracy: {accuracy['accuracy']}\n"

print(f"Accuracy = {accuracy['accuracy']}")

Overwriting accelerate_scripts/03_accelerate_base_code_more_bs.py


He quitado los prints extra, porque ya hemos visto que el código se está ejecutando en las dos GPUs y he aunmentado el batch size de 64 a 128. Lo ejecutamos a ver

In [28]:
%%time

!accelerate launch accelerate_scripts/03_accelerate_base_code_more_bs.py

Accuracy = 0.1052                                                               
Accuracy = 0.1052
CPU times: user 1.41 s, sys: 180 ms, total: 1.59 s
Wall time: 2min 22s


Aumentando el batch size ha bajado unos segundos el tiempo de ejecución

### Ejecución de procesos

#### Ejecución de código en un único proceso

Antes hemos visto que los `print`s se imprimían dos veces, esto es porque `accelerate` crea tantos procesos como dispositivos donde se ejecuta el código, en mi caso crea dos procesos por tener dos GPUs.

Sin embargo no todo el código debería ejecutarse en todos los procesos, por ejemplo los `print`s, ralentizan mucho el código, como para ejecutarlo varias veces, si se guardan los checkpoints, se guardarían dos veces, etc.

Para poder ejecutar parte de un código en un único proceso se tiene que encapsular en una función y decorarla con `accelerator.on_local_main_process`, por ejemplo en el siguiente código vas a ver que he creado la siguiente función

``` python
@accelerator.on_local_main_process
def print_something(something):
    print(something)
```

Otra opción es meter el código dentro de un `if accelerator.is_local_main_process` como en el siguiente código

``` python
if accelerator.is_local_main_process:
    print("Something")
```

In [61]:
%%writefile accelerate_scripts/04_accelerate_base_code_some_code_in_one_process.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from fastprogress.fastprogress import master_bar, progress_bar

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

@accelerator.on_local_main_process
def print_something(something):
    print(something)

master_progress_bar = master_bar(range(EPOCHS))
for i in master_progress_bar:
    model.train()
    progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        master_progress_bar.child.comment = f'loss: {loss}'

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
    master_progress_bar.main_bar.comment = f"Validation accuracy: {accuracy['accuracy']}\n"

# print(f"Accuracy = {accuracy['accuracy']}")
print_something(f"Accuracy = {accuracy['accuracy']}")

if accelerator.is_local_main_process:
    print(f"End of script with {accuracy['accuracy']} accuracy")

Overwriting accelerate_scripts/04_accelerate_base_code_some_code_in_one_process.py


Vamos a ejecutarlo a ver

In [62]:
%%time

!accelerate launch accelerate_scripts/04_accelerate_base_code_some_code_in_one_process.py

Accuracy = 0.2098                                                               
End of script with 0.2098 accuracy
CPU times: user 1.38 s, sys: 197 ms, total: 1.58 s
Wall time: 2min 22s


Ahora solo se ha impreso el print una vez

Sin embargo, aunque no se ve mucho, las barras de progreso se ejecutan en cada proceso.

No he encontrado una manera de evitar esto con las barras de progreso de `fastprogress`, pero sí con las de `tqdm`, así que voy a sustituir las barras de progreso de `fastprogress` por las de `tqdm` y para que se ejecuten en un único proceso hay que añadirle el argumento `disable=not accelerator.is_local_main_process`

In [4]:
%%writefile accelerate_scripts/05_accelerate_base_code_some_code_in_one_process.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

@accelerator.on_local_main_process
def print_something(something):
    print(something)

for i in range(EPOCHS):
    model.train()
    # progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        # master_progress_bar.child.comment = f'loss: {loss}'

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    # progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
# print(f"Accuracy = {accuracy['accuracy']}")
print_something(f"Accuracy = {accuracy['accuracy']}")

if accelerator.is_local_main_process:
    print(f"End of script with {accuracy['accuracy']} accuracy")

Overwriting accelerate_scripts/05_accelerate_base_code_some_code_in_one_process.py


In [1]:
%%time

!accelerate launch accelerate_scripts/05_accelerate_base_code_some_code_in_one_process.py

100%|███████████████████████████████████████████| 20/20 [00:06<00:00,  3.30it/s]
Accuracy = 0.2166
End of script with 0.2166 accuracy
CPU times: user 1.33 s, sys: 195 ms, total: 1.52 s
Wall time: 2min 22s


Hemos mostrado un ejemplo de cómo imprimir en un solo proceso, y esto ha sido una manera de ejecutar procesos en un solo proceso. Pero si lo que quieres es solo imprimir en un solo proceso se puede usar el método `print` de `accelerate`. Vamos a ver el mismo ejemplo de antes con este método

In [1]:
%%writefile accelerate_scripts/06_accelerate_base_code_print_one_process.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

for i in range(EPOCHS):
    model.train()
    # progress_bar_train = progress_bar(dataloader["train"], parent=master_progress_bar)
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)
        # master_progress_bar.child.comment = f'loss: {loss}'

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    # progress_bar_validation = progress_bar(dataloader["validation"], parent=master_progress_bar)
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
# print(f"Accuracy = {accuracy['accuracy']}")
accelerator.print(f"Accuracy = {accuracy['accuracy']}")

if accelerator.is_local_main_process:
    print(f"End of script with {accuracy['accuracy']} accuracy")

Writing accelerate_scripts/06_accelerate_base_code_print_one_process.py


Lo ejecutamos

In [2]:
%%time

!accelerate launch accelerate_scripts/06_accelerate_base_code_print_one_process.py

100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.33it/s]
Accuracy = 0.2134
End of script with 0.2134 accuracy
CPU times: user 1.4 s, sys: 189 ms, total: 1.59 s
Wall time: 2min 27s


#### Ejecución de código en todos los procesos

Sin embargo hay código que debe ejecutarse en todos los procesos, por ejemplo si subimos los checkpoints al hub, así que aquí tenemos dos opciones, encapsular el código en una función y decorarla con `accelerator.on_main_process`

``` python
@accelerator.on_main_process
def do_my_thing():
    "Something done once per server"
    do_thing_once()
```

o meter el código dentro de un `if accelerator.is_main_process`

``` python
if accelerator.is_main_process:
    repo.push_to_hub()
```

Como estamos haciendo entrenamientos solo para mostrar la librería `accelerate` y el modelo que estamos entrenando no es bueno, no tiene sentido ahora subir los checkpoints al hub, así que voy a hacer un ejemplo con `print`s

In [12]:
%%writefile accelerate_scripts/07_accelerate_base_code_some_code_in_all_process.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

@accelerator.on_local_main_process
def print_in_one_process(something):
    print(something)

@accelerator.on_main_process
def print_in_all_processes(something):
    print(something)

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
print_in_one_process(f"Accuracy = {accuracy['accuracy']}")

if accelerator.is_local_main_process:
    print(f"End of script with {accuracy['accuracy']} accuracy")

print_in_all_processes(f"All process: Accuracy = {accuracy['accuracy']}")

if accelerator.is_main_process:
    print(f"All process: End of script with {accuracy['accuracy']} accuracy")

Overwriting accelerate_scripts/06_accelerate_base_code_some_code_in_all_process.py


Lo ejecutamos a ver

In [9]:
%%time

!accelerate launch accelerate_scripts/07_accelerate_base_code_some_code_in_all_process.py

100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.34it/s]
Accuracy = 0.2092
End of script with 0.2092 accuracy
All process: Accuracy = 0.2092
All process: End of script with 0.2092 accuracy
CPU times: user 1.42 s, sys: 216 ms, total: 1.64 s
Wall time: 2min 27s


#### Ejecución de código en el proceso X

Por último podemos especificar en qué proceso queremos ejecutar código, para esto hay que crear una función y decorarla con `@accelerator.on_process(process_index=0)`

``` python
@accelerator.on_process(process_index=0)
def do_my_thing():
    "Something done on process index 0"
    do_thing_on_index_zero()
```

o decorarla con `@accelerator.on_local_process(local_process_idx=0)`

``` python
@accelerator.on_local_process(local_process_index=0)
def do_my_thing():
    "Something done on process index 0 on each server"
    do_thing_on_index_zero_on_each_server()
```

Aquí he puesto el proceso 0, pero se puede poner cualquier número

In [18]:
%%writefile accelerate_scripts/08_accelerate_base_code_some_code_in_some_process.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

@accelerator.on_local_main_process
def print_in_one_process(something):
    print(something)

@accelerator.on_main_process
def print_in_all_processes(something):
    print(something)

@accelerator.on_process(process_index=0)
def print_in_process_0(something):
    print("Process 0: " + something)

@accelerator.on_local_process(local_process_index=1)
def print_in_process_1(something):
    print("Process 1: " + something)

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
print_in_one_process(f"Accuracy = {accuracy['accuracy']}")

if accelerator.is_local_main_process:
    print(f"End of script with {accuracy['accuracy']} accuracy")

print_in_all_processes(f"All process: Accuracy = {accuracy['accuracy']}")

if accelerator.is_main_process:
    print(f"All process: End of script with {accuracy['accuracy']} accuracy")

print_in_process_0("End of process 0")
print_in_process_1("End of process 1")

Overwriting accelerate_scripts/07_accelerate_base_code_some_code_in_some_process.py


Lo ejecutamos

In [15]:
%%time

!accelerate launch accelerate_scripts/08_accelerate_base_code_some_code_in_some_process.py

100%|███████████████████████████████████████████| 20/20 [00:06<00:00,  3.27it/s]
Process 1: End of process 1
Accuracy = 0.2128
End of script with 0.2128 accuracy
All process: Accuracy = 0.2128
All process: End of script with 0.2128 accuracy
Process 0: End of process 0
CPU times: user 1.42 s, sys: 295 ms, total: 1.71 s
Wall time: 2min 37s


#### Sincronizar procesos

Si tenemos código que debe ejecutarse en todos los procesos, es interesante esperar a que termine en todos los procesos antes de hacer otra tarea, así que para ello usamos `accelerator.wait_for_everyone()`

Para verlo vamos a meter un retardo en una de las funciones de imprimir en un proceso

Además he puesto un break en el bucle de entrenamiento para que no esté mucho tiempo entrenando, que no es lo que ahora nos interesa

In [22]:
%%writefile accelerate_scripts/09_accelerate_base_code_sync_all_process.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm
import time

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

@accelerator.on_local_main_process
def print_in_one_process(something):
    print(something)

@accelerator.on_main_process
def print_in_all_processes(something):
    print(something)

@accelerator.on_process(process_index=0)
def print_in_process_0(something):
    time.sleep(2)
    print("Process 0: " + something)

@accelerator.on_local_process(local_process_index=1)
def print_in_process_1(something):
    print("Process 1: " + something)

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()
        break

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
print_in_one_process(f"Accuracy = {accuracy['accuracy']}")

if accelerator.is_local_main_process:
    print(f"End of script with {accuracy['accuracy']} accuracy")

print_in_all_processes(f"All process: Accuracy = {accuracy['accuracy']}")

if accelerator.is_main_process:
    print(f"All process: End of script with {accuracy['accuracy']} accuracy")

print_in_one_process("Printing with delay in process 0")
print_in_process_0("End of process 0")
print_in_process_1("End of process 1")
accelerator.wait_for_everyone()

print_in_one_process("End of script")

Overwriting accelerate_scripts/08_accelerate_base_code_sync_all_process.py


Lo ejecutamos

In [23]:
!accelerate launch accelerate_scripts/09_accelerate_base_code_sync_all_process.py

100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.58it/s]
Process 1: End of process 1
Accuracy = 0.212
End of script with 0.212 accuracy
All process: Accuracy = 0.212
All process: End of script with 0.212 accuracy
Printing with delay in process 0
Process 0: End of process 0
End of script


Como se puede ver primero se ha impreso `Process 1: End of process 1` y luego el resto, esto es porque el resto de prints se hacen o en el proceso 0 o en todos los procesos, así que hasta que no termine el delay de 2 segundos que hemos puesto no se ejecuta el resto de código

### Guardar y cargar el state dict

Cuando entrenamos, a veces guardamos el estado para poder seguir en otro momento

Para guardar el estado tendremos que usar los métodos `save_state()` y `load_state()`

In [66]:
%%writefile accelerate_scripts/10_accelerate_save_and_load_checkpoints.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

@accelerator.on_local_main_process
def print_something(something):
    print(something)

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()

    # Guardamos los pesos
    accelerator.save_state("accelerate_scripts/checkpoints")

print_something(f"Accuracy = {accuracy['accuracy']}")

# Cargamos los pesos
accelerator.load_state("accelerate_scripts/checkpoints")

Overwriting accelerate_scripts/09_accelerate_save_and_load_checkpoints.py


Lo ejecutamos

In [67]:
!accelerate launch accelerate_scripts/10_accelerate_save_and_load_checkpoints.py

100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.40it/s]
Accuracy = 0.2142


### Guardar el modelo

Cuando se usó el método `prepare` se envolvió el modelo para poder guardarlo en los dispositivos necesarios. Por lo que a la hora de guardarlo tenemos que usar el método `save_model` que primero lo desenvuelve y luego lo guarda. Además si usamos el parámetro `safe_serialization=True` se guardará el modelo como un `safe tensor`

In [1]:
%%writefile accelerate_scripts/11_accelerate_save_model.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

@accelerator.on_local_main_process
def print_something(something):
    print(something)

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()

    # Guardamos el modelo
    accelerator.wait_for_everyone()
    accelerator.save_model(model, "accelerate_scripts/model", safe_serialization=True)

print_something(f"Accuracy = {accuracy['accuracy']}")

Writing accelerate_scripts/11_accelerate_save_model.py


Lo ejecutamos

In [78]:
!accelerate launch accelerate_scripts/11_accelerate_save_model.py

100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.35it/s]
Accuracy = 0.214


### Guardar el modelo `pretrained`

En modelos que usan la librería `transformers` debemos guardar el modelo con el método `save_pretrained` para poder cargarlo con el método `from_pretrained`. Antes de guardarlo hay que desenvolverlo con el método `unwrap_model`

In [79]:
%%writefile accelerate_scripts/12_accelerate_save_pretrained.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

@accelerator.on_local_main_process
def print_something(something):
    print(something)

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()

    # Guardamos el modelo pretrained
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(
        "accelerate_scripts/model_pretrained",
        is_main_process=accelerator.is_main_process,
        save_function=accelerator.save,
    )

print_something(f"Accuracy = {accuracy['accuracy']}")

Writing accelerate_scripts/11_accelerate_save_pretrained.py


Lo ejecutamos

In [80]:
!accelerate launch accelerate_scripts/12_accelerate_save_pretrained.py

100%|███████████████████████████████████████████| 20/20 [00:05<00:00,  3.37it/s]
Accuracy = 0.21


Ahora lo podríamos cargar

In [82]:
from transformers import AutoModel

checkpoints = "accelerate_scripts/model_pretrained"
tokenizer = AutoModel.from_pretrained(checkpoints)

Some weights of RobertaModel were not initialized from the model checkpoint at accelerate_scripts/model_pretrained and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Entrenamiento en notebooks

Hasta ahora hemos visto cómo ejecutar scripts, pero si quieres ejecutar el código en un notebook, podemos escribir el mismo código de antes, pero encapsulado en una función

Primero importamos las librerás

In [1]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm
import time
# from accelerate import Accelerator

Ahora creamos la función

In [2]:
def train_code(batch_size: int = 64):
    from accelerate import Accelerator
    accelerator = Accelerator()

    dataset = load_dataset("tweet_eval", "emoji")
    num_classes = len(dataset["train"].info.features["label"].names)
    max_len = 130

    checkpoints = "cardiffnlp/twitter-roberta-base-irony"
    tokenizer = AutoTokenizer.from_pretrained(checkpoints)

    def tokenize_function(dataset):
        return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
    tokenized_dataset = {
        "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
        "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
        "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
    }
    tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
    tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
    tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

    BS = batch_size
    dataloader = {
        "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
        "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
        "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
    }

    model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
    model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    metric = evaluate.load("accuracy")

    EPOCHS = 1
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    device = accelerator.device

    # model.to(device)
    model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

    for i in range(EPOCHS):
        model.train()
        progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
        for batch in progress_bar_train:
            optimizer.zero_grad()

            input_ids = batch["input_ids"]#.to(device)
            attention_mask = batch["attention_mask"]#.to(device)
            labels = batch["label"]#.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_function(outputs['logits'], labels)

            # loss.backward()
            accelerator.backward(loss)
            optimizer.step()

        model.eval()
        progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
        for batch in progress_bar_validation:
            input_ids = batch["input_ids"]#.to(device)
            attention_mask = batch["attention_mask"]#.to(device)
            labels = batch["label"]#.to(device)

            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            predictions = torch.argmax(outputs['logits'], axis=-1)
            # Recopilamos las predicciones de todos los dispositivos
            predictions = accelerator.gather_for_metrics(predictions)
            labels = accelerator.gather_for_metrics(labels)

            accuracy = metric.add_batch(predictions=predictions, references=labels)
        accuracy = metric.compute()
        
    accelerator.print(f"Accuracy = {accuracy['accuracy']}")

Para poder ejecutar el entrenamiento en el notebook usamos la funcion `notebook_launcher`, al que le pasamos la función que queremos ejecutar, los argumentos de esa función y el número de GPUs en las que vamos a entrenar con la variable `num_processes`

In [10]:
from accelerate import notebook_launcher

args = (128,)
notebook_launcher(train_code, args, num_processes=2)

Launching training on 2 GPUs.


100%|██████████| 20/20 [00:06<00:00,  3.31it/s]


Accuracy = 0.2112


### Entrenamiento en FP16

Cuando al principio configuramos `accelerate` nos preguntó `Do you wish to use FP16 or BF16 (mixed precision)?` y dijimos que no, así que ahora vamos a decirle que sí, que queremos en FP16

Hasta ahora hemos entrenado en FP32, lo que quiere decir que cada peso del modelo es un número en coma flotante de 32 bits, y ahora vamos a usar un número en coma flotante de 16 bits, es decir, el modelo va a ocupar menos. Por lo que van a pasar dos cosas, podremos usar un batch size mayor y además será más rápido

Primero volvemos a lanzar `accelerate config` y le vamos a decir que queremos FP16

In [4]:
!accelerate config

--------------------------------------------------------------------------------
In which compute environment are you running?
This machine
--------------------------------------------------------------------------------
multi-GPU
How many different machines will you use (use more than 1 for multi-node training)? [1]: 1
Should distributed operations be checked while running for errors? This can avoid timeout issues but will be slower. [yes/NO]: no
Do you wish to optimize your script with torch dynamo?[yes/NO]:no
Do you want to use DeepSpeed? [yes/NO]: no
Do you want to use FullyShardedDataParallel? [yes/NO]: no
Do you want to use Megatron-LM ? [yes/NO]: no
How many GPU(s) should be used for distributed training? [1]:2
What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:0,1
--------------------------------------------------------------------------------
Do you wish to use FP16 or BF16 (mixed precision)?
fp16
accelerate configuration saved at 

Ahora creamos un script para entrenar, con el mismo batch size de antes, para ver si tarda menos en entrenar

In [2]:
%%writefile accelerate_scripts/13_accelerate_base_code_fp16_bs128.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 128
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
accelerator.print(f"Accuracy = {accuracy['accuracy']}")

Overwriting accelerate_scripts/12_accelerate_base_code_fp16_bs128.py


Lo ejecutamos a ver cuanto tarda

In [3]:
%%time

!accelerate launch accelerate_scripts/13_accelerate_base_code_fp16_bs128.py

100%|███████████████████████████████████████████| 20/20 [00:02<00:00,  6.84it/s]
Accuracy = 0.2094
CPU times: user 812 ms, sys: 163 ms, total: 976 ms
Wall time: 1min 27s


Cuando ejecutamos este entrenamiento en FP32 tardó unos 2 minutos y medio, y ahora más o menos 1 minuto y medio. Vamos a ver si ahora en vez de entrenar con un batch size de 128, lo hacemos con uno de 256

In [6]:
%%writefile accelerate_scripts/14_accelerate_base_code_fp16_bs256.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 256
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
accelerator.print(f"Accuracy = {accuracy['accuracy']}")

Overwriting accelerate_scripts/13_accelerate_base_code_fp16_bs256.py


Lo ejecutamos

In [7]:
%%time

!accelerate launch accelerate_scripts/14_accelerate_base_code_fp16_bs256.py

100%|███████████████████████████████████████████| 10/10 [00:02<00:00,  3.45it/s]
Accuracy = 0.2236
CPU times: user 670 ms, sys: 91.6 ms, total: 761 ms
Wall time: 1min 12s


Ha bajado solo unos 15 segundos

### Entrenamiento en BF16

Antes hemos entrenado en FP16 y ahora lo vamos a hacer en BF16, ¿Cuál es la diferencia?

![FP32_FP16_BF16](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/FP32_FP16_BF16.webp)

Como podemos ver en la imagen, mientras que FP16 en comparación con FP32 tiene menos bits en la mantisa y el exponente, lo que hace que su rango sea mucho menor, BF16 en comparación con FP32 tiene el mismo número de bits del exponente pero menos en la mantisa, lo que hace que BF16 tenga el mismo rango de números que FP32, pero es menos preciso

Esto es beneficioso porque en FP16 algunos cálculos podrían dar números muy altos, que en formato FP16 no se podrían representar. Además hay ciertos dispositivos HW que están optimizados para este formato

Al igual que antes ejecutamos `accelerate config` y le indicamos que queremos BF16

In [8]:
!accelerate config

--------------------------------------------------------------------------------
In which compute environment are you running?
This machine
--------------------------------------------------------------------------------
multi-GPU
How many different machines will you use (use more than 1 for multi-node training)? [1]: 1
Should distributed operations be checked while running for errors? This can avoid timeout issues but will be slower. [yes/NO]: no
Do you wish to optimize your script with torch dynamo?[yes/NO]:no
Do you want to use DeepSpeed? [yes/NO]: no
Do you want to use FullyShardedDataParallel? [yes/NO]: no
Do you want to use Megatron-LM ? [yes/NO]: no
How many GPU(s) should be used for distributed training? [1]:2
What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:0,1
--------------------------------------------------------------------------------
Do you wish to use FP16 or BF16 (mixed precision)?
bf16
accelerate configuration saved at 

Ahora ejecutamos el último script que habíamos creado, es decir con un batch size de 256

In [9]:
%%time

!accelerate launch accelerate_scripts/14_accelerate_base_code_fp16_bs256.py

100%|███████████████████████████████████████████| 10/10 [00:03<00:00,  3.21it/s]
Accuracy = 0.2112
CPU times: user 688 ms, sys: 144 ms, total: 832 ms
Wall time: 1min 17s


Ha tardado un tiempo similar a lo que tardó antes, lo cual es normal, ya que hemos entrenado un modelo con pesos de 16 bits, al igual que antes

### Entrenamiento en FP8

Ahora vamos a entrenar en formato FP8, que como su nombre indica, es un formato de coma flotante, donde cada peso tiene 8 bits, por lo que ejecutamos `accelerate config` para decirle que queremos FP8

In [10]:
!accelerate config

--------------------------------------------------------------------------------
In which compute environment are you running?
This machine
--------------------------------------------------------------------------------
multi-GPU
How many different machines will you use (use more than 1 for multi-node training)? [1]: 1
Should distributed operations be checked while running for errors? This can avoid timeout issues but will be slower. [yes/NO]: no
Do you wish to optimize your script with torch dynamo?[yes/NO]:no
Do you want to use DeepSpeed? [yes/NO]: no
Do you want to use FullyShardedDataParallel? [yes/NO]: no
Do you want to use Megatron-LM ? [yes/NO]: no
How many GPU(s) should be used for distributed training? [1]:2
What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:0,1
--------------------------------------------------------------------------------
Do you wish to use FP16 or BF16 (mixed precision)?
fp8
accelerate configuration saved at ~

Ahora ejecutamos el último script, el de batch size de 256

In [11]:
%%time

!accelerate launch accelerate_scripts/14_accelerate_base_code_fp16_bs256.py

Traceback (most recent call last):
  File "/home/wallabot/Documentos/web/portafolio/posts/accelerate_scripts/13_accelerate_base_code_fp16_bs256.py", line 12, in <module>
    accelerator = Accelerator()
                  ^^^^^^^^^^^^^
  File "/home/wallabot/miniconda3/envs/nlp/lib/python3.11/site-packages/accelerate/accelerator.py", line 371, in __init__
    self.state = AcceleratorState(
                 ^^^^^^^^^^^^^^^^^
  File "/home/wallabot/miniconda3/envs/nlp/lib/python3.11/site-packages/accelerate/state.py", line 790, in __init__
    raise ValueError(
ValueError: Using `fp8` precision requires `transformer_engine` or `MS-AMP` to be installed.
Traceback (most recent call last):
  File "/home/wallabot/Documentos/web/portafolio/posts/accelerate_scripts/13_accelerate_base_code_fp16_bs256.py", line 12, in <module>
    accelerator = Accelerator()
                  ^^^^^^^^^^^^^
  File "/home/wallabot/miniconda3/envs/nlp/lib/python3.11/site-packages/accelerate/accelerator.py", line 371,

Como los pesos ahora son de 8 bits y ocupan la mitad de memoria vamos a subir el batch size a 512

In [2]:
%%writefile accelerate_scripts/15_accelerate_base_code_fp8_bs512.py

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import tqdm

# Importamos e inicializamos Accelerator
from accelerate import Accelerator
accelerator = Accelerator()

dataset = load_dataset("tweet_eval", "emoji")
num_classes = len(dataset["train"].info.features["label"].names)
max_len = 130

checkpoints = "cardiffnlp/twitter-roberta-base-irony"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

def tokenize_function(dataset):
    return tokenizer(dataset["text"], max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
tokenized_dataset = {
    "train": dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "validation": dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"]),
    "test": dataset["test"].map(tokenize_function, batched=True, remove_columns=["text"]),
}
tokenized_dataset["train"].set_format(type="torch", columns=['input_ids', 'attention_mask', 'label'])
tokenized_dataset["validation"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["test"].set_format(type="torch", columns=['label', 'input_ids', 'attention_mask'])

BS = 512
dataloader = {
    "train": DataLoader(tokenized_dataset["train"], batch_size=BS, shuffle=True),
    "validation": DataLoader(tokenized_dataset["validation"], batch_size=BS, shuffle=True),
    "test": DataLoader(tokenized_dataset["test"], batch_size=BS, shuffle=True),
}

model = AutoModelForSequenceClassification.from_pretrained(checkpoints)
model.classifier.out_proj = torch.nn.Linear(in_features=model.classifier.out_proj.in_features, out_features=num_classes, bias=True)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=5e-4)
metric = evaluate.load("accuracy")

EPOCHS = 1
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = accelerator.device

# model.to(device)
model, optimizer, dataloader["train"], dataloader["validation"] = accelerator.prepare(model, optimizer, dataloader["train"], dataloader["validation"])

for i in range(EPOCHS):
    model.train()
    progress_bar_train = tqdm.tqdm(dataloader["train"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_train:
        optimizer.zero_grad()

        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_function(outputs['logits'], labels)

        # loss.backward()
        accelerator.backward(loss)
        optimizer.step()

    model.eval()
    progress_bar_validation = tqdm.tqdm(dataloader["validation"], disable=not accelerator.is_local_main_process)
    for batch in progress_bar_validation:
        input_ids = batch["input_ids"]#.to(device)
        attention_mask = batch["attention_mask"]#.to(device)
        labels = batch["label"]#.to(device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs['logits'], axis=-1)
        # Recopilamos las predicciones de todos los dispositivos
        predictions = accelerator.gather_for_metrics(predictions)
        labels = accelerator.gather_for_metrics(labels)

        accuracy = metric.add_batch(predictions=predictions, references=labels)
    accuracy = metric.compute()
    
accelerator.print(f"Accuracy = {accuracy['accuracy']}")

Writing accelerate_scripts/15_accelerate_base_code_fp8_bs512.py


Lo ejecutamos

In [ ]:
%%time

!accelerate launch accelerate_scripts/15_accelerate_base_code_fp8_bs512.py

## Inferencia de modelos

### Uso del ecosistema de Hugging Face

Vamos a ver cómo hacer inferencia de grandes modelos con la librería `transformers` de hugging face.

#### Inferencia con `pipeline`

Si usamos el ecosistema de Hugging Face es muy sencillo, ya que todo se produce por debajo sin tener que hacer nosotros mucho. En el caso de usar `pipeline`, que es la manera más sencilla de hacer inferencia con la librería `transformers`, simplemente tenemos que decirle el modelo que queremos usar y muy importante, pasarle `device_map="auto"`. Esto hará que por debajo `accelerate` distribuya el modelo entre las distintas GPUs, RAM de la CPU o disco duro si es necesario

Hay más posibles valores para `device_map`, que los veremos más adelante, pero de momento quédate con `"auto"`.

Vamos a usar el modelo `Llama3 8B`, que como su nombre indica es un modelo de unos 8 mil millones de parámetros, como cada parámetro por defecto está en formato FP32, que corresponde a 4 bytes (32 bits), eso quiere decir que si multiplicamos 8 mil millones de parámetros por 4 bytes, nos queda que necesitaría una GPU con unos 32 GB de VRAM.

En mi caso tengo 2 GPUs de 24 GB de VRAM, por lo que no entraría en una sola GPU. Pero gracias a poner `device_map="auto"`, accelerate desitribuirá el modelo entre las dos GPUs y podré realizar la inferencia

In [ ]:
%%writefile accelerate_scripts/16_inference_with_pipeline.py

from transformers import pipeline

checkpoints = "meta-llama/Meta-Llama-3-8B-Instruct"
generator = pipeline(model=checkpoints, device_map="auto")

prompt = "Conoces accelerate de hugging face?"
output = generator(prompt)
print(output)

Overwriting accelerate_scripts/09_inference_with_pipeline.py


Ahora lo ejecutamos, solo que como pipeline usa por debajo accelerate, no necesitamos ejecutarlo con `acelerate launch script.py` sino que con `python script.py` vale

In [ ]:
!python accelerate_scripts/16_inference_with_pipeline.py

Loading checkpoint shards: 100%|██████████████████| 4/4 [00:09<00:00,  2.27s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
[{'generated_text': 'Conoces accelerate de hugging face? ¿Qué es el modelo de lenguaje de transformers y cómo se utiliza en el marco de hugging face? ¿Cómo puedo utilizar modelos de lenguaje de transformers en mi aplicación? ¿Qué son los tokenizers y cómo se utilizan en el marco de hugging face? ¿Cómo puedo crear un modelo de lenguaje personalizado utilizando transformers y hugging face? ¿Qué son los datasets y cómo se utilizan en el marco de hugging face? ¿Cómo puedo utilizar datasets para entrenar un modelo de lenguaje personalizado utilizando transformers y hugging face? ¿Qué son los evaluadores y cómo se utilizan en el marco de hugging face? ¿Cómo puedo utilizar evaluadores para evaluar el rendimiento de un modelo 

Como se puede ver, no ha respondido, sino que ha seguido haciendo preguntas. Esto es porque Llama3 es un modelo de lenguaje que lo que hace es predecir el siguiente token, así que con el prompt que le he pasado, ha considerado que los siguientes mejores tokens son unos que corresponden a más preguntas. Lo cual tiene sentido, porque hay veces que la gente tiene dudas sobre un tema y genera muchas preguntas, así que para que nos conteste a la pregunta hay que condicionarle un poco

In [ ]:
%%writefile accelerate_scripts/17_inference_with_pipeline_condition.py

from transformers import pipeline

checkpoints = "meta-llama/Meta-Llama-3-8B-Instruct"
generator = pipeline(model=checkpoints, device_map="auto")

prompt = "Conoces accelerate de hugging face?"
messages = [
    {
        "role": "system",
        "content": "Eres un chatbot amigable que siempre intenta solucionar las dudas",
    },
    {"role": "user", "content": f"{prompt}"},
]
output = generator(messages)
print(output[0]['generated_text'][-1])

Overwriting accelerate_scripts/10_inference_with_pipeline_condition.py


Como ves se ha generado un mensaje con roles, condicionando el modelo y con el prompt

In [ ]:
!python accelerate_scripts/17_inference_with_pipeline_condition.py

Loading checkpoint shards: 100%|██████████████████| 4/4 [00:09<00:00,  2.41s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
{'role': 'assistant', 'content': '¡Hola!\n\nSí, conozco Accelerate de Hugging Face. Accelerate es una biblioteca de Python desarrollada por Hugging Face que se enfoca en simplificar y acelerar el entrenamiento y la evaluación de modelos de lenguaje en diferentes dispositivos y entornos.\n\nCon Accelerate, puedes entrenar modelos de lenguaje en diferentes plataformas y dispositivos, como GPUs, TPUs, CPUs y servidores, sin necesidad de cambiar el código de tu modelo. Esto te permite aprovechar al máximo la potencia de cálculo de tus dispositivos y reducir el tiempo de entrenamiento.\n\nAccelerate también ofrece varias características adicionales, como:\n\n* Soporte para diferentes frameworks de machine learning, como Ten

Ahora la respuesta si responde nuestro prompt

#### Inferencia con `AutoClass`

Por último vamos a ver cómo hacer la inferencia solo con `AutoClass`.

In [ ]:
%%writefile accelerate_scripts/18_inference_with_autoclass.py

from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

checkpoints = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(checkpoints, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(checkpoints, device_map="auto")
streamer = TextStreamer(tokenizer)

prompt = "Conoces accelerate de hugging face?"
tokens_input = tokenizer([prompt], return_tensors="pt").to(model.device)

_ = model.generate(**tokens_input, streamer=streamer, max_new_tokens=500, do_sample=True, top_k=50, top_p=0.95, temperature=0.7)

Overwriting accelerate_scripts/11_inference_with_autoclass.py


Como se puede ver, se ha creado el objeto `streamer` que luego se le pasa al método `generate` del modelo. Esto es útil para que se vaya imprimiendo cada palabra a medida que se va generando y no haya que esperar a que se genere toda la salida para imprimirla

In [ ]:
!python accelerate_scripts/18_inference_with_autoclass.py

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:09<00:00,  2.28s/it]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
<|begin_of_text|>Conoces accelerate de hugging face? Si es así, puedes utilizar la biblioteca `transformers` de Hugging Face para crear un modelo de lenguaje que pueda predecir la siguiente palabra en una secuencia de texto.

Aquí te muestro un ejemplo de cómo hacerlo:
```
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Cargar el modelo y el tokenizador
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Cargar el conjunto de datos
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Preprocesar los datos
train_texts = train_df["t

### Uso pytorch

Normalmente la manera de hacer inferencias con pytorch es crear un modelo con los pesos inicializados aleatoriamente y a continuación cargar un `state dict` con los pesos del modelo preentrenado, así que para obtener ese `state dict` vamos a hacer primero una pequeña trampa y nos los vamos a descargar

In [ ]:
import torch
import torchvision.models as models

model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
torch.save(model.state_dict(), 'accelerate_scripts/resnet152_pretrained.pth')

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /home/maximo.fernandez/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [02:48<00:00, 1.43MB/s] 


Ahora que tenemos el `state dict` vamos a hacer inferencia como se hace normalmente en pytorch

In [ ]:
import torch
import torchvision.models as models

device = "cuda" if torch.cuda.is_available() else "cpu"     # Set device

resnet152 = models.resnet152().to(device) # Create model with random weights and move to device
state_dict = torch.load('accelerate_scripts/resnet152_pretrained.pth', map_location=device) # Load pretrained weights into device memory
resnet152.load_state_dict(state_dict) # Load this weights into the model

input = torch.rand(1, 3, 224, 224).to(device)  # Random image with batch size 1
output = resnet152(input)
output.shape

torch.Size([1, 1000])

Vamos a explicar qué ha pasado

 * Cuando hemos hecho `resnet152 = models.resnet152().to(device)` se ha cargado una resnet152 con pesos aleatorios en la memoria de la GPU
 * Cuando hemos hecho `state_dict = torch.load('accelerate_scripts/resnet152_pretrained.pth', map_location=device)` se ha cargado un diccionario con los pesos entrenados en la memoria de la GPU
 * Cuando hemos hecho `resnet152.load_state_dict(state_dict)` se han asignado esos pesos preentrenados al modelo

Es decir se ha cargado dos veces el modelo en la memoria de la GPU

Te puedes estar preguntando por qué hemos hecho primero 

``` python
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
torch.save(model.state_dict(), 'accelerate_scripts/resnet152_pretrained.pth')
```

Para luego hacer

``` python
resnet152 = models.resnet152().to(device)
state_dict = torch.load('accelerate_scripts/resnet152_pretrained.pth', map_location=device)
resnet152.load_state_dict(state_dict)
```

Y por que no usamos directamente

```
model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
```

Y nos dejamos de guardar el `state dict` para luego cargarlo. Bueno, pues porque Pytorch, por edbajo hace lo mismo que hemos hecho. Así que para poder ver todo el proceso hemos hecho en varias lineas lo que Pytorch hace en una

Esta manera de trabajar ha funcionado bien hasta ahora, mientras que los modelos tenían un tamaño manejable por las GPUs de usuario. Pero desde la llegada de los LLMs este enfoque no tiene sentido

Por ejemplo un modelo de 6B de parámetros ocuparía en la memoria 24 GB, y como se carga dos veces con esta manera de trabajar haría falta tener una GPU de 48 GB.

Así que para arreglar esto, la manera de cargar un modelo preentrenado de Pytorch es:
 * Crear un modelo vacío con `init_empty_weights` que no ocupará RAM
 * Luego cargar los pesos con `load_checkpoint_and_dispatch` que cargará un punto de control dentro del modelo vacío y distribuirá los pesos para cada capa en todos los dispositivos que se tenga disponibles (GPU, CPU RAM y disco duro), gracias a poner `device_map="auto"`

In [ ]:
import torch
import torchvision.models as models
from accelerate import init_empty_weights, load_checkpoint_and_dispatch

with init_empty_weights():
    resnet152 = models.resnet152()

resnet152 = load_checkpoint_and_dispatch(resnet152, checkpoint='accelerate_scripts/resnet152_pretrained.pth', device_map="auto")

device = "cuda" if torch.cuda.is_available() else "cpu"

input = torch.rand(1, 3, 224, 224).to(device)  # Random image with batch size 1
output = resnet152(input)
output.shape

torch.Size([1, 1000])

### Cómo funciona accelerate por debajo

En este vídeo se puede ver gráficamente cómo funciona accelerate por debajo

<iframe width="1280" height="720" src="https://www.youtube.com/embed/MWCSGj9jEAo" title="Accelerate Big Model Inference: How Does it Work?" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

#### Inicialización de un modelo vacío

`Accelerate` crea el esqueleto de un modelo vacío mediante `init_empty_weights` para que ocupe la menor cantidad de memoria posible

Por ejemplo, veamos cuanta RAM tengo ahora disponible en mi ordenador

In [ ]:
import psutil

def get_ram_info():
    ram = dict(psutil.virtual_memory()._asdict())
    print(f"Total RAM: {(ram['total']/1024/1024/1024):.2f} GB, Available RAM: {(ram['available']/1024/1024/1024):.2f} GB, Used RAM: {(ram['used']/1024/1024/1024):.2f} GB")

get_ram_info()

Total RAM: 31.24 GB, Available RAM: 22.62 GB, Used RAM: 7.82 GB


Tengo unos 22 GB de RAM disponibles

Ahora vamos a intentar crear un modelo 5000x1000x1000 parámetros, es decir de 5B de parámetros, si cada parámetro está en FP32, supone 20 GB de RAM

In [ ]:
import torch
from torch import nn

model = nn.Sequential(*[nn.Linear(5000, 1000) for _ in range(1000)])

Si volvemos a ver la RAM

In [ ]:
get_ram_info()

Total RAM: 31.24 GB, Available RAM: 3.77 GB, Used RAM: 26.70 GB


Como vemos ahora solo tenemos 3 GB de RAM disponibles

Ahora vamos a eliminar el modelo para liberar RAM

In [ ]:
del model
get_ram_info()

Total RAM: 31.24 GB, Available RAM: 22.44 GB, Used RAM: 8.03 GB


Volvemos a tener unos 22 GB de RAM disponibles

Vamos ahora a usar `init_empty_weights` de `accelerate` y luego vemos la RAM

In [ ]:
from accelerate import init_empty_weights

with init_empty_weights():
    model = nn.Sequential(*[nn.Linear(5000, 1000) for _ in range(1000)])

get_ram_info()

Total RAM: 31.24 GB, Available RAM: 22.32 GB, Used RAM: 8.16 GB


Antes teníamos exactamente 22.44 GB libres y tras crear el modelo con `init_empty_weights` tenemos 22.32 GB. El ahorro en RAM es enorme! Casi no se ha ocupado RAM para crear el modelo.

Esto se basa en el metadispositivo introducido en PyTorch 1.9, por lo que es importante que para usar `accelerate` tengamos una versión de Pytorch posterior

#### Carga de los pesos

Una vez hemos inicializado el modelo tenemos que cargarle los pesos que lo hacemos mediante `load_checkpoint_and_dispatch` que como su nombre indica carga los pesos y los envía al dispositivo o dispositivos que sea necesario